In [9]:
import pandas as pd 
import emoji

In [10]:
gpt = pd.read_json('../../datasets/gpt35.json', lines=True)

In [11]:
gpt.drop("timestamp", axis=1, inplace=True)
gpt.drop("users", axis=1, inplace=True)

In [12]:
gpt.head(10)

,tweets,artificial
0,Just joined this urgent call for climate actio...,1
1,Did you know that indigenous and local communi...,1
2,Secretary Yellen highlights the urgency of add...,1
3,@coolfunnytshirt @BefittingFacts The smog towe...,1
4,"The U.N. climate summit in Glasgow has begun, ...",1
5,"I do hope #COP26 from/inGlasgow, is a great on...",0
6,'Moving forward on #ClimateAction and sustaina...,1
7,Children and youth must be a central focus at ...,1
8,"The image may be false, but @JaredHuffman's ba...",1
9,"Hey @cdnmedia, how about a media personality d...",1


In [13]:
#delete Emojis
def delete_emojis(text):
    return emoji.demojize(text)


gpt["tweets"] = gpt["tweets"].apply(delete_emojis)


In [14]:
#remove URL
gpt['tweets'] = gpt['tweets'].str.replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)


In [15]:
#remove taggings    
gpt['tweets'] = gpt['tweets'].str.replace(r'@\S+', '', regex=True)

In [16]:
#remove #
gpt['tweets'] = gpt['tweets'].str.replace(r'#\S+', '', regex=True)


In [17]:
#remove quotes
gpt['tweets'] = gpt['tweets'].str.replace(r'"', '', regex=True).replace(r"'", '', regex=True)

In [18]:
#remove every special characters
gpt['tweets'] = gpt['tweets'].str.replace(r'[^\w\s]', '', regex=True)

In [19]:
#remove numbers
gpt['tweets'] = gpt['tweets'].str.replace(r'\d+', '', regex=True)

In [20]:
#lowercase
gpt['tweets'] = gpt['tweets'].str.lower()

In [21]:
#remove stopwords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop = stopwords.words('english')

gpt["tweets"] = gpt["tweets"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
#lemmatize everything
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.tag import pos_tag

# Download necessary ntkl data files
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(text):
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in tagged_tokens]
    return ' '.join(lemmatized_tokens)

gpt["tweets"] = gpt["tweets"].apply(lemmatize_text)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/romanoelfken/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [23]:
#change column name
gpt.rename(columns={"tweets": "text"}, inplace=True)
gpt.rename(columns={"artificial": "label"}, inplace=True)

In [24]:
#writing to csv
gpt.to_csv('../StorePreprocessed/GPTcsv.csv', index=False)